In [ ]:
# Import the required modules
from pyesgf.search import SearchConnection
import os
import sys
import glob
import importlib
import pandas as pd
import requests
from tqdm import tqdm

# Set the os environment to on
os.environ['ESGF_PYCLIENT_NO_FACETS_STAR_WARNING'] = "on"

In [ ]:
# Import the functions
sys.path.append('/home/users/benhutch/downloading-data/DAMIP/')

# Import the functions
from testing_download_functions import *

In [ ]:
# Import the dictionaries
import dictionaries as dicts

In [ ]:
# Import the functions again, so as to not have to restart the kernel
importlib.reload(sys.modules['testing_download_functions'])

# Import the functions again
from testing_download_functions import find_valid_nodes, create_results_list, \
                                        extract_file_context, check_file_exists_jasmin

# Import the dictionaries again, so as to not have to restart the kernel
importlib.reload(sys.modules['dictionaries'])

# Import the dictionaries again
import dictionaries as dicts

In [ ]:
# # Set up the directory for lookin in
# old_dir = "/gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/data/pr/MPI-ESM1-2-LR/"

# # Set up the target directory
# target_dir = "/gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/pr/MPI-ESM1-2-LR/"

# # If the target directory does not exist, create it
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)

# # First an dlast years
# first_year = 1960
# last_year = 2018

# # Set the number of ensemble members
# nens = 16

# # Loop over the years
# for year in range(first_year, last_year + 1):
#     print("Year: " + str(year))
    
#     # Verify that there are 20 files containing s{year} in the directory
#     # Set up the directory
#     files = os.listdir(old_dir)

#     # Set up the filenames
#     filenames = "*s" + str(year) + "*.nc"

#     # Check that there are 20 files
#     matching_files = glob.glob(old_dir + filenames)

#     # Assert that there are 20 files
#     if len(matching_files) != nens:
#         print("There are not 20 files for year " + str(year))
#         continue

In [ ]:
# Set up the search connection
connection = SearchConnection(dicts.search_connection, distrib=True)

In [ ]:
# Set up the paramas
# To speed up the check, only search for year s1961
params = {
    'activity_id': 'DCPP',
    'experiment_id': 'dcppA-hindcast',
    'latest': True,
    'project': 'CMIP6',
    'table_id': 'Amon',
    'sub_experiment_id': ['s1960', 's1961', 's1962', 's1963', 's1964', 's1965', 's1966', 's1967', 's1968', 's1969', 's1970', 's1971', 's1972', 's1973', 's1974', 's1975', 's1976', 's1977', 's1978', 's1979', 's1980', 's1981', 's1982', 's1983', 's1984', 's1985', 's1986', 's1987', 's1988', 's1989', 's1990', 's1991', 's1992', 's1993', 's1994', 's1995', 's1996', 's1997', 's1998', 's1999', 's2000', 's2001', 's2002', 's2003', 's2004', 's2005', 's2006', 's2007', 's2008', 's2009', 's2010', 's2011', 's2012', 's2013', 's2014', 's2015', 's2016', 's2017', 's2018']
}

In [ ]:
# Loop over the variables
# variables = dicts.variables
models = ["CNRM-ESM2-1"]

# Set up the test variable - 'pr' precip
variables = ['psl']

# Create a dataframe for the results
# Containing three columns: model, experiment, does_model_exist
results_df = pd.DataFrame(columns=['model', 'variable', 'does_model_exist'])

# Loop over the variables
for variable in variables:
    print("Checking whether models exist for variable: " + variable)

    # Loop over the models
    for model in models:
        print("Checking whether model: " + model + " exists for variable: " + variable)

        # Set up the params
        params['variable_id'] = variable
        params['source_id'] = model

        # Query the database
        ctx = connection.new_context(**params)

        try:
            # Get the results from the query
            results = ctx.search()
        except:
            print("Model: " + model + " does not exist for variable: " + variable)
            results = []

        # If the length of results is greater than 0
        if len(results) > 0:
            print("Model: " + model + " exists for variable: " + variable)

            # Append True to the dataframe
            results_df = pd.concat([results_df, pd.DataFrame({'model': [model], 'variable': [variable], 'does_model_exist': [True]})], ignore_index=True)
        else:
            print("Model: " + model + " does not exist for variable: " + variable)

            # Append False to the dataframe
            results_df = pd.concat([results_df, pd.DataFrame({'model': [model], 'variable': [variable], 'does_model_exist': [False]})], ignore_index=True)

# Print the dataframe
results_df

In [ ]:
# Create an empty dictionary to store the valid nodes for each variable
variable_nodes = {}


# Find the valid nodes for each variable and models list combination
# Loop over the var models dictionary
for var, models_list in dicts.var_models_test_psl_CNRM_ESM2_1.items():
    print("Finding the valid nodes for variable: " + var)
    print("Models list: " + str(models_list))

    # Set up the variable_id for the params
    params['variable_id'] = var

    # Append the variable to the dictionary
    variable_nodes[var] = {}

    # Find the valid nodes
    valid_nodes = find_valid_nodes(params=params, 
                                   models_list=models_list,
                                   conn=connection)
    
    # Append the valid nodes to the dictionary
    variable_nodes[var] = valid_nodes


In [ ]:
# # Modify the data node in variable_nodes[rsds] to be 'esgf-data1.llnl.gov'
variable_nodes['psl'][0]['data_node'] = 'esg1.umr-cnrm.fr'

In [ ]:
# Print the variable_nodes dictionary

variable_nodes['psl']

In [ ]:
import importlib

importlib.reload(sys.modules['testing_download_functions'])

# Import the functions again
from testing_download_functions import find_valid_nodes, create_results_list, \
                                        extract_file_context, check_file_exists_jasmin

In [ ]:
# For each variable, create the results list
# Initialize an empty dictionary to store the results
results = {}

# Loop over the variables
for var in ['psl']:
    print("Finding the results for variable: " + var)

    # Extract the valid nodes for the variable
    valid_nodes = variable_nodes[var]

    # Append the variable to params
    params['variable_id'] = var

    # print the type of valid nodes
    print("Valid nodes type: " + str(type(valid_nodes)))
    print("Valid nodes: " + str(valid_nodes))

    # Find the results for the variable
    var_results = create_results_list(params=params,
                                      max_results_list=valid_nodes,
                                      connection=connection)

    # Append the results to the dictionary
    results[var] = var_results

In [ ]:
print("Results: " + str(results))
# print the length of the results
print("Length of results: " + str(len(results)))
print(type(results['psl']))

In [ ]:
# Initialize an empty dictionary to store the results file context
results_file_context = {}

# For each of the variables
for var, results in results.items():
    print("Variable: " + var)
    print("Results: " + str(results))

    # Initialize an empty dictionary to store the file context
    file_context_list = []

    # Loop over the results
    for result in results:
        print("Result: " + str(result))

        # Extract the file context
        file_context = extract_file_context(result)

        # Append the file context to the list
        file_context_list.append(file_context)

    # Append the file context list to the dictionary
    results_file_context[var] = file_context_list

In [ ]:
results_file_context['psl']

In [ ]:
# Create an empty dataframe to store the results
df = pd.DataFrame()

for var, file_context_lists in results_file_context.items():
    print("Variable: " + var)

    # Loop over the file contexts
    for file_context in file_context_lists:
        # Convert the dictionary to a dataframe
        file_context_df = pd.DataFrame.from_dict(file_context)

        # Add a new column on the far left of the dataframe
        # containing the variable name
        file_context_df.insert(0, 'variable', var)

        # Concatenate the dataframe to the results dataframe
        df = pd.concat([df, file_context_df], ignore_index=True)

# Print the dataframe
df

In [ ]:
# Extract the dictionaries in the first row of the dataframe into a new dataframe
# containing only the first row, with headers 'filename' and 'url'
# {'filename': 'pr_Amon_MIROC6_dcppA-hindcast_s1992-r2i1p1f1_gn_199211-200212.nc', 'url': 'http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/DCPP/MIROC/MIROC6/dcppA-hindcast/s1992-r2i1p1f1/Amon/pr/gn/v20190821/pr_Amon_MIROC6_dcppA-hindcast_s1992-r2i1p1f1_gn_199211-200212.nc'}

In [ ]:
# create a copy of the dataframe
df_copy = df.copy()

# Extract the first row of the dataframe
first_row = df_copy.iloc[0]

# Print the first row
type(first_row)

# Convert the first row to a dataframe
first_row_df = pd.DataFrame(first_row).T

# # Print the first row dataframe
# first_row_df

# Remove the 'variable' column from the first row dataframe
first_row_df = first_row_df.drop(columns='variable')

# Remove the file_exists and filepaths columns from the first row dataframe
# first_row_df = first_row_df.drop(columns='file_exists')

# first_row_df = first_row_df.drop(columns='filepath')

first_row_df

# Set up a list of filenames
filenames = [ ]
urls = [ ]

# Loop over the columns in the first row
for column in first_row_df.columns:
    # print("Column: " + column)

    # If the cell contains a dictionary
    if type(first_row_df[column].values[0]) == dict:
        print("Cell is a dictionary")

        # Extract the filename and url from the dictionary
        filename = first_row_df[column].values[0]['filename']
        url = first_row_df[column].values[0]['url']

        # Append the filename and url to the lists
        filenames.append(filename)
        urls.append(url)

# # Loop over the columns in the first row
# for column in first_row.columns:
#     print("Column: " + column)

#     # If the cell contains a dictionary
#     if type(first_row[column]) == dict:
#         print("Cell is a dictionary")

#         # Extract the filename and url from the dictionary
#         filename = first_row[column]['filename']
#         url = first_row[column]['url']

#         # Append the filename and url to the lists
#         filenames.append(filename)
#         urls.append(url)

In [ ]:
# Set up a new dataframe containing the filenames and urls
df_filenames_urls = pd.DataFrame({'filename': filenames, 'url': urls})

# Print the dataframe
df_filenames_urls

In [ ]:
# # Set up the directory for the data on JASMIN
# dcpp_dir_badc = "/badc/cmip6/data/CMIP6/DCPP/"

# # Checkk whether these files exist on JASMIN
# jasmin_files_df = check_file_exists_jasmin(df=df_filenames_urls,
#                                             directory=dcpp_dir_badc)

In [ ]:
import importlib

importlib.reload(sys.modules['testing_download_functions'])

# Import the functions again
from testing_download_functions import find_valid_nodes, create_results_list, \
                                        extract_file_context, check_file_exists_jasmin

In [ ]:
# Now set up the group work space directory
dcpp_dir_gws = "/gws/nopw/j04/canari/users/benhutch/"

# Check whether these files exist on JAASMIN
jasmin_files_df_gws = check_file_exists_jasmin(df=df_filenames_urls,
                                            directory=dcpp_dir_gws) 

In [ ]:
jasmin_files_df_gws

In [ ]:
new_dir_df = jasmin_files_df_gws.copy()

# Verify that there are 10 files for available for each year s{year}
# from 1960 to 2021
# Loop over the years
for year in range(1960, 2015):
    print("Year: " + str(year))

    # find all of the rows with filenames containing s{year}
    # and print the length of the dataframe
    print("Length of dataframe: " + str(len(new_dir_df[new_dir_df['filename'].str.contains("s" + str(year))])))

    # Assert that the length of the dataframe is 10
    assert len(new_dir_df[new_dir_df['filename'].str.contains("s" + str(year))]) == 20

In [ ]:
# if file exist = True, check the size of the file and append this 'file_size'
# to the dataframe
# Set up an empty list to store the file sizes
file_sizes = []

# Loop over the rows in the dataframe
for index, row in jasmin_files_df_gws.iterrows():
    print("Index: " + str(index))
    print("Row: " + str(row))

    # If the file exists
    if row['file_exists'] == True:
        print("File exists")

        # Set up the file path
        file_path = row['filepath']

        # Get the size of the file
        file_size = os.path.getsize(file_path)

        # Append the file size to the list
        file_sizes.append(file_size)
    else:
        print("File does not exist")

        # Append None to the list
        file_sizes.append(None)

In [ ]:
# /gws/nopw/j04/canari/users/benhutch/pr_Amon_MIROC6_dcppA-hindcast_s1991-r8i1p1f1_gn_199111-200112.nc - doesn't exist
# /gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/data/pr/MIROC6/pr_Amon_MIROC6_dcppA-hindcast_s1991-r8i1p1f1_gn_199111-200112.nc
# /gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/data/pr/MIROC6/pr_Amon_MIROC6_dcppA-hindcast_s1991-r8i1p1f1_gn_199111-200112.nc

In [ ]:
jasmin_files_df_gws['file_size'] = file_sizes

In [ ]:
jasmin_files_df_gws

In [ ]:
# Is the filesize is less than 1,000,000 bytes, then the file is too small
# Remove the filepath value for that row
# and set the file_exists value to False
# Loop over the rows in the dataframe
for index, row in jasmin_files_df_gws.iterrows():
    print("Index: " + str(index))
    print("Row: " + str(row))

    # If the file size is less than 1,000,000 bytes
    if row['file_size'] < 1000000:
        print("File size is less than 1,000,000 bytes")

        # Set the file_exists value to False
        jasmin_files_df_gws.at[index, 'file_exists'] = False

        # Set the filepath value to None
        jasmin_files_df_gws.at[index, 'filepath'] = None

In [ ]:
jasmin_files_df_gws

In [ ]:
importlib.reload(sys.modules['testing_download_functions'])

# Import the functions again
from testing_download_functions import download_files

In [ ]:
# Test the download_files function
# TODO: Reduce the number of print statements used in this
download_files(download_dir=dicts.dcpp_dir_gws,
               df=jasmin_files_df_gws
)

In [ ]:
jasmin_files_df_gws

In [ ]:
# Now we want to verify that the files have been downloaded
# Although some of them are in /gws/nopw/j04/canari/users/benhutch + dcppA-hindcast/pr/MIROC6/
# While others are in /gws/nopw/j04/canari/users/benhutch + dcppA-hindcast/data/pr/MIROC6/

# We are going to use the /gws/nopw/j04/canari/users/benhutch + dcppA-hindcast/pr/MIROC6/
# as the main data store
# But first we need to move the files from /gws/nopw/j04/canari/users/benhutch + dcppA-hindcast/data/pr/MIROC6/
# to /gws/nopw/j04/canari/users/benhutch + dcppA-hindcast/pr/MIROC6/
# BUt first we want to remove the files in /gws/nopw/j04/canari/users/benhutch + dcppA-hindcast/data/pr/MIROC6/
# Which are less than 1,000,000 bytes
old_dir = "/gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/data/rsds/CMCC-CM2-SR5/"

# Set up an empty list for the filenames
filenames = []
file_sizes = []

# In the old directory, check the file sizes
# Loop over the files in the directory
for file in os.listdir(old_dir):
    print("File: " + file)

    # Set up the file path
    file_path = old_dir + file

    # Get the size of the file
    file_size = os.path.getsize(file_path)

    # Append the file size to the list
    file_sizes.append(file_size)

    # Append the filename to the list
    filenames.append(file)

# Set up a dataframe containing the filenames and file sizes
old_dir_df = pd.DataFrame({'filename': filenames, 'file_size': file_sizes})

In [ ]:
# Create a new column in the dataframe
# a boolen called 'remove_file'
# which is True if the file size is less than 1,000,000 bytes
old_dir_df['remove_file'] = old_dir_df['file_size'] < 1000000

In [ ]:
old_dir_df

In [ ]:
# Remove the files from the old directory which are less than 1,000,000 bytes
# Loop over the rows in the dataframe
for index, row in old_dir_df.iterrows():
    print("Index: " + str(index))
    print("Row: " + str(row))

    # If the file size is less than 1,000,000 bytes
    if row['remove_file'] == True:
        print("File size is less than 1,000,000 bytes")

        # Set up the file path
        file_path = old_dir + row['filename']

        # Remove the file
        os.remove(file_path)

In [ ]:
# MOve the files from the old directory to the new directory
new_dir = "/gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/rsds/CMCC-CM2-SR5/"

# Loop over the files in the old directory
for file in os.listdir(old_dir):
    print("File: " + file)

    # Set up the old file path
    old_file_path = old_dir + file

    # Set up the new file path
    new_file_path = new_dir + file

    # Move the file
    os.rename(old_file_path, new_file_path)

In [ ]:
# Now create a new dataframe containing the filenames and file sizes
new_dir_filenames = []
new_dir_file_sizes = []

# Loop over the files in the new directory
for file in os.listdir(new_dir):
    print("File: " + file)

    # Set up the file path
    file_path = new_dir + file

    # Get the size of the file
    file_size = os.path.getsize(file_path)

    # Append the file size to the list
    new_dir_file_sizes.append(file_size)

    # Append the filename to the list
    new_dir_filenames.append(file)

# Set up a dataframe containing the filenames and file sizes
new_dir_df = pd.DataFrame({'filename': new_dir_filenames, 'file_size': new_dir_file_sizes})

In [ ]:
new_dir_df

In [ ]:
# Now we want to copy the files in /badc/cmip6/data/CMIP6/DCPP/CMCC/CMCC-CM2-SR5/dcppA-hindcast/Amon/rsds/gn/v20190710/
# to the /gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/rsds/CMCC-CM2-SR5/ directory
# /badc/cmip6/data/CMIP6/DCPP/CMCC/CMCC-CM2-SR5/dcppA-hindcast/s1962-r1i1p1f1/Amon/rsds/gn/files/d20210805/rsds_Amon_CMCC-CM2-SR5_dcppA-hindcast_s1962-r1i1p1f1_gn_196211-197212.nc
# Split the filename column by "_" and extract the 4th element
patterns = []

# Loop over the filenames
for filename in new_dir_df['filename']:
    print("Filename: " + filename)

    # Split the filename by "_"
    split_filename = filename.split("_")

    # Append the 4th element to the list
    patterns.append(split_filename[4])

In [ ]:
# Split patterns by "-"
init_years = [pattern.split("-")[0][1:] for pattern in patterns]

# split patterns by "-"
ens_members = [pattern.split("-")[1] for pattern in patterns]

# Extract the list of unique initialisation years
unique_init_years = list(set(init_years))

# Extract the list of unique ensemble members
unique_ens_members = list(set(ens_members))

In [ ]:
# Print the size of the unique initialisation years
print("Size of unique initialisation years: " + str(len(unique_init_years)))

# Print the size of the unique ensemble members
print("Size of unique ensemble members: " + str((unique_ens_members)))

In [ ]:
# Add the initialisation years and ensemble members to the dataframe
new_dir_df['init_year'] = init_years
new_dir_df['ens_member'] = ens_members

In [ ]:
new_dir_df

In [ ]:
# Each init_year should have 20 ensemble members, one for each of the unique ensemble members
# Add new rows into the dataframe for the missing ensemble members
# Loop over the unique initialisation years
for init_year in unique_init_years:
    print("Init year: " + init_year)

    # Loop over the unique ensemble members
    for ens_member in unique_ens_members:
        print("Ens member: " + ens_member)

        # If the init_year and ens_member combination is not in the dataframe
        if (new_dir_df['init_year'] == init_year).sum() != 20:
            print("Init year and ens member combination is not in the dataframe")

            # Set up a new row
            new_row = {'filename': None, 'file_size': None, 'init_year': init_year, 'ens_member': ens_member}

            # Append the new row to the dataframe
            new_dir_df = new_dir_df.append(new_row, ignore_index=True)

In [ ]:
new_dir_df

In [ ]:
# Create a new column in the dataframe - "file_path"
# containing the file path for each file
# Loop over the rows in the dataframe
for index, row in new_dir_df.iterrows():
    print("Index: " + str(index))
    print("Row: " + str(row))

    # Set up the file path
    file_path = new_dir + str(row['filename'])

    # Set the file path in the dataframe
    new_dir_df.at[index, 'file_path'] = file_path

In [ ]:
new_dir_df

In [ ]:
# Where filename and file_size are not None, check whether the file exists
# Create a new column in the dataframe - "file_exists"
# containing a boolean value for whether the file exists
# Loop over the rows in the dataframe
for index, row in new_dir_df.iterrows():
    print("Index: " + str(index))
    print("Row: " + str(row))

    # If the filename and file_size are not None
    if (row['filename'] != None) & (row['file_size'] != None):
        print("Filename and file size are not None")

        # Check whether the file exists
        file_exists = os.path.exists(row['file_path'])

        # Set the file_exists value in the dataframe
        new_dir_df.at[index, 'file_exists'] = file_exists

In [ ]:
new_dir_df

In [ ]:
# Remove the rows which contain ensemble member r11-r20
# Loop over the rows in the dataframe
# Constrain the unique ensemble members list
# loop over the unique ensemble members
for unique_ens_member in unique_ens_members:
    print("Unique ensemble member: " + unique_ens_member)

    # If the unique ensemble member contains r11-r20
    for i in range(11, 21):
        print("i: " + str(i))

        # Set up the ensemble member
        ens_member = "r" + str(i) + "i1p1f1"

        # If the unique ensemble member contains the ensemble member  
        if unique_ens_member == ens_member:
            print("Unique ensemble member contains ensemble member: " + ens_member)

            # Remove the rows which contain the ensemble member
            new_dir_df = new_dir_df[new_dir_df['ens_member'] != ens_member]

In [ ]:
new_dir_df

In [ ]:
import glob

# Where file_exists is not True, we want to find the file on JASMIN within badc
# loop over the rows in the dataframe
for index, row in new_dir_df.iterrows():
    print("Index: " + str(index))
    print("Row: " + str(row))

    # If the file exists value is not True
    if row['file_exists'] != True:
        print("File exists value is not True")

        # Set up the filename
        filename = row['filename']

        # Set up the directory
        # /badc/cmip6/data/CMIP6/DCPP/CMCC/CMCC-CM2-SR5/dcppA-hindcast/s1962-r1i1p1f1/Amon/rsds/gn/files/d20210805/
        files = f"/badc/cmip6/data/CMIP6/DCPP/CMCC/CMCC-CM2-SR5/dcppA-hindcast/s{row['init_year']}-{row['ens_member']}/Amon/rsds/gn/files/*/*.nc"

        # print the files
        print("Files: " + files)

        # Get the file path
        file_path = glob.glob(files)


        # Asert that the length of the file path is 1
        if len(file_path) != 1:
            print("Length of file path is not 1")
            print("File path: " + str(file_path))

            # Assertion eroor
            # Set the file exists value in the dataframe
            new_dir_df.at[index, 'file_exists'] = False

            # Set the file path in the dataframe
            new_dir_df.at[index, 'file_path'] = None
        else:

            # Set the file path in the dataframe
            new_dir_df.at[index, 'file_path'] = file_path[0]

            # Set the file exists value in the dataframe
            new_dir_df.at[index, 'file_exists'] = True

In [ ]:
new_dir_df

In [ ]:
# Verify that there are 10 files for available for each year s{year}
# from 1960 to 2021
# Loop over the years
for year in range(1960, 2022):
    print("Year: " + str(year))

    # find all of the rows with filenames containing s{year}
    # and print the length of the dataframe
    print("Length of dataframe: " + str(len(new_dir_df[new_dir_df['filename'].str.contains("s" + str(year))])))

    # Assert that the length of the dataframe is 10
    assert len(new_dir_df[new_dir_df['filename'].str.contains("s" + str(year))]) == 10